In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim import Adam, SGD
import torchvision
import torchvision.transforms as transforms

import sys, os, math, csv
import argparse
import pandas as pd

In [2]:
lr=0.01
data='cifar10'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar10_vgg_Swish.pth'
resume = False

In [3]:
if data == 'cifar10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR10(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR10(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)
    
elif data == 'cifar100':
    nclass = 100
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR100(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR100(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)
    
elif data == 'stl10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.Resize((img_width,img_width)),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.Resize((img_width,img_width)),
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.STL10(root=root, split='train', transform=transform_train, target_transform=None, download=True)
    testset = torchvision.datasets.STL10(root=root, split='test', transform=transform_test, target_transform=None, download=True)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
if model == 'vgg':
    from models.vgg_dist import VGG_Dist
    net = nn.DataParallel(VGG_Dist(nclass, img_width=img_width).cuda())
    
net

DataParallel(
  (module): VGG_Dist(
    (conv1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv4): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_

In [5]:
if resume:
    print(f'==> Resuming from {model_out}')
    net.load_state_dict(torch.load(model_out))

In [6]:
cudnn.benchmark = True

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
writefile_train = 'train.csv'
fieldnames = ['Epochs', 'Train Accuracy']
with open( writefile_train, 'w' ) as f:
    writer = csv.writer(f)
    writer.writerow(fieldnames)

In [9]:
def train(epoch):
    print('Epoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        pred = torch.max(outputs, dim=1)[1]
        correct += torch.sum(pred.eq(targets)).item()
        total += targets.numel()
    print(f'[TRAIN] Acc: {100.*correct/total:.3f}')
    with open( writefile_train, 'a' ) as f:
            writer = csv.writer(f)
            writer.writerow([epoch, 100.*correct/total])

In [10]:
writefile_test = 'test.csv'
fieldnames = ['Epochs', 'Test Accuracy']
with open( writefile_test, 'w' ) as f:
    writer = csv.writer(f)
    writer.writerow(fieldnames)

In [11]:
def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print(f'[TEST] Acc: {100.*correct/total:.3f}')
        
        with open( writefile_test, 'a' ) as f:
            writer = csv.writer(f)
            writer.writerow([epoch, 100.*correct/total])

    # Save checkpoint after each epoch
    torch.save(net.state_dict(), model_out)

In [12]:
if data == 'cifar10':
    epochs = [50, 50, 50, 50]
elif data == 'cifar100':
    epochs = [50, 50, 50, 50]
elif data == 'stl10':
    epochs = [50, 50, 50, 50]

In [13]:
count = 0

In [14]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0


/home/kumar/yes/envs/bayesian/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[TRAIN] Acc: 23.042
[TEST] Acc: 33.400
Epoch: 1
[TRAIN] Acc: 40.658
[TEST] Acc: 46.360
Epoch: 2
[TRAIN] Acc: 59.158
[TEST] Acc: 40.020
Epoch: 3
[TRAIN] Acc: 68.018
[TEST] Acc: 67.170
Epoch: 4
[TRAIN] Acc: 73.438
[TEST] Acc: 66.740
Epoch: 5
[TRAIN] Acc: 78.038
[TEST] Acc: 73.800
Epoch: 6
[TRAIN] Acc: 81.504
[TEST] Acc: 75.960
Epoch: 7
[TRAIN] Acc: 83.980
[TEST] Acc: 79.980
Epoch: 8
[TRAIN] Acc: 86.276
[TEST] Acc: 79.010
Epoch: 9
[TRAIN] Acc: 88.652
[TEST] Acc: 78.450
Epoch: 10
[TRAIN] Acc: 90.190
[TEST] Acc: 76.730
Epoch: 11
[TRAIN] Acc: 91.702
[TEST] Acc: 76.770
Epoch: 12
[TRAIN] Acc: 93.150
[TEST] Acc: 78.390
Epoch: 13
[TRAIN] Acc: 94.002
[TEST] Acc: 79.600
Epoch: 14
[TRAIN] Acc: 95.074
[TEST] Acc: 79.430
Epoch: 15
[TRAIN] Acc: 95.956
[TEST] Acc: 81.410
Epoch: 16
[TRAIN] Acc: 96.336
[TEST] Acc: 81.280
Epoch: 17
[TRAIN] Acc: 96.744
[TEST] Acc: 82.430
Epoch: 18
[TRAIN] Acc: 97.340
[TEST] Acc: 83.480
Epoch: 19
[TRAIN] Acc: 97.538
[TEST] Acc: 83.360
Epoch: 20
[TRAIN] Acc: 97.738
[TEST] Ac

[TEST] Acc: 86.570
Epoch: 166
[TRAIN] Acc: 100.000
[TEST] Acc: 86.530
Epoch: 167
[TRAIN] Acc: 100.000
[TEST] Acc: 86.350
Epoch: 168
[TRAIN] Acc: 100.000
[TEST] Acc: 86.390
Epoch: 169
[TRAIN] Acc: 100.000
[TEST] Acc: 86.520
Epoch: 170
[TRAIN] Acc: 100.000
[TEST] Acc: 86.500
Epoch: 171
[TRAIN] Acc: 100.000
[TEST] Acc: 86.440
Epoch: 172
[TRAIN] Acc: 100.000
[TEST] Acc: 86.550
Epoch: 173
[TRAIN] Acc: 100.000
[TEST] Acc: 86.520
Epoch: 174
[TRAIN] Acc: 100.000
[TEST] Acc: 86.570
Epoch: 175
[TRAIN] Acc: 100.000
[TEST] Acc: 86.450
Epoch: 176
[TRAIN] Acc: 100.000
[TEST] Acc: 86.590
Epoch: 177
[TRAIN] Acc: 100.000
[TEST] Acc: 86.600
Epoch: 178
[TRAIN] Acc: 100.000
[TEST] Acc: 86.430
Epoch: 179
[TRAIN] Acc: 100.000
[TEST] Acc: 86.540
Epoch: 180
[TRAIN] Acc: 100.000
[TEST] Acc: 86.500
Epoch: 181
[TRAIN] Acc: 100.000
[TEST] Acc: 86.420
Epoch: 182
[TRAIN] Acc: 100.000
[TEST] Acc: 86.390
Epoch: 183
[TRAIN] Acc: 100.000
[TEST] Acc: 86.570
Epoch: 184
[TRAIN] Acc: 100.000
[TEST] Acc: 86.570
Epoch: 185
[

In [15]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

merged=test.merge(train[['Epochs', 'Train Accuracy']], how='left', left_on='Epochs', right_on='Epochs')

merged.to_csv('VGG_CIFAR10_Swish.csv', index=False)

In [16]:
os.remove("train.csv")
os.remove("test.csv")

In [2]:
lr=0.01
data='stl10'
root='./data/'
model='vgg'
model_out='./checkpoint/stl10_vgg_Swish.pth'
resume = False

In [14]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0


/home/kumar/yes/envs/bayesian/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[TRAIN] Acc: 9.860
[TEST] Acc: 10.088
Epoch: 1
[TRAIN] Acc: 12.020
[TEST] Acc: 17.700
Epoch: 2
[TRAIN] Acc: 17.420
[TEST] Acc: 13.213
Epoch: 3
[TRAIN] Acc: 21.060
[TEST] Acc: 16.087
Epoch: 4
[TRAIN] Acc: 22.780
[TEST] Acc: 21.788
Epoch: 5
[TRAIN] Acc: 25.120
[TEST] Acc: 16.788
Epoch: 6
[TRAIN] Acc: 28.260
[TEST] Acc: 29.262
Epoch: 7
[TRAIN] Acc: 29.800
[TEST] Acc: 29.812
Epoch: 8
[TRAIN] Acc: 32.640
[TEST] Acc: 34.438
Epoch: 9
[TRAIN] Acc: 35.940
[TEST] Acc: 22.212
Epoch: 10
[TRAIN] Acc: 35.640
[TEST] Acc: 31.550
Epoch: 11
[TRAIN] Acc: 37.560
[TEST] Acc: 37.025
Epoch: 12
[TRAIN] Acc: 39.560
[TEST] Acc: 38.438
Epoch: 13
[TRAIN] Acc: 41.140
[TEST] Acc: 35.600
Epoch: 14
[TRAIN] Acc: 41.600
[TEST] Acc: 37.638
Epoch: 15
[TRAIN] Acc: 42.620
[TEST] Acc: 32.750
Epoch: 16
[TRAIN] Acc: 44.320
[TEST] Acc: 40.000
Epoch: 17
[TRAIN] Acc: 48.300
[TEST] Acc: 45.263
Epoch: 18
[TRAIN] Acc: 50.040
[TEST] Acc: 29.538
Epoch: 19
[TRAIN] Acc: 52.160
[TEST] Acc: 44.013
Epoch: 20
[TRAIN] Acc: 54.140
[TEST] Acc

[TEST] Acc: 63.300
Epoch: 165
[TRAIN] Acc: 100.000
[TEST] Acc: 63.138
Epoch: 166
[TRAIN] Acc: 100.000
[TEST] Acc: 63.212
Epoch: 167
[TRAIN] Acc: 100.000
[TEST] Acc: 63.163
Epoch: 168
[TRAIN] Acc: 100.000
[TEST] Acc: 63.288
Epoch: 169
[TRAIN] Acc: 100.000
[TEST] Acc: 63.362
Epoch: 170
[TRAIN] Acc: 100.000
[TEST] Acc: 63.288
Epoch: 171
[TRAIN] Acc: 100.000
[TEST] Acc: 63.237
Epoch: 172
[TRAIN] Acc: 100.000
[TEST] Acc: 63.337
Epoch: 173
[TRAIN] Acc: 100.000
[TEST] Acc: 63.337
Epoch: 174
[TRAIN] Acc: 100.000
[TEST] Acc: 63.288
Epoch: 175
[TRAIN] Acc: 100.000
[TEST] Acc: 63.237
Epoch: 176
[TRAIN] Acc: 100.000
[TEST] Acc: 63.388
Epoch: 177
[TRAIN] Acc: 100.000
[TEST] Acc: 63.263
Epoch: 178
[TRAIN] Acc: 100.000
[TEST] Acc: 63.200
Epoch: 179
[TRAIN] Acc: 100.000
[TEST] Acc: 63.163
Epoch: 180
[TRAIN] Acc: 100.000
[TEST] Acc: 63.212
Epoch: 181
[TRAIN] Acc: 100.000
[TEST] Acc: 63.212
Epoch: 182
[TRAIN] Acc: 100.000
[TEST] Acc: 63.375
Epoch: 183
[TRAIN] Acc: 100.000
[TEST] Acc: 63.325
Epoch: 184
[

In [15]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

merged=test.merge(train[['Epochs', 'Train Accuracy']], how='left', left_on='Epochs', right_on='Epochs')

merged.to_csv('VGG_STL10_Swish.csv', index=False)

In [16]:
os.remove("train.csv")
os.remove("test.csv")

In [17]:
# print("Model's state_dict:")
# for param_tensor in net.state_dict():
#     print(param_tensor, "\t", net.state_dict()[param_tensor].size())

In [2]:
lr=0.01
data='cifar100'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar100_vgg_Swish.pth'
resume = False

In [14]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0


/home/kumar/yes/envs/bayesian/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[TRAIN] Acc: 2.304
[TEST] Acc: 3.580
Epoch: 1
[TRAIN] Acc: 4.436
[TEST] Acc: 6.380
Epoch: 2
[TRAIN] Acc: 7.378
[TEST] Acc: 6.570
Epoch: 3
[TRAIN] Acc: 10.628
[TEST] Acc: 11.460
Epoch: 4
[TRAIN] Acc: 14.946
[TEST] Acc: 16.190
Epoch: 5
[TRAIN] Acc: 20.224
[TEST] Acc: 20.420
Epoch: 6
[TRAIN] Acc: 26.354
[TEST] Acc: 24.400
Epoch: 7
[TRAIN] Acc: 32.542
[TEST] Acc: 28.640
Epoch: 8
[TRAIN] Acc: 37.520
[TEST] Acc: 33.950
Epoch: 9
[TRAIN] Acc: 42.250
[TEST] Acc: 38.430
Epoch: 10
[TRAIN] Acc: 47.350
[TEST] Acc: 41.560
Epoch: 11
[TRAIN] Acc: 51.480
[TEST] Acc: 42.790
Epoch: 12
[TRAIN] Acc: 56.082
[TEST] Acc: 42.590
Epoch: 13
[TRAIN] Acc: 60.992
[TEST] Acc: 44.810
Epoch: 14
[TRAIN] Acc: 65.258
[TEST] Acc: 46.800
Epoch: 15
[TRAIN] Acc: 69.706
[TEST] Acc: 47.560
Epoch: 16
[TRAIN] Acc: 73.634
[TEST] Acc: 47.550
Epoch: 17
[TRAIN] Acc: 77.950
[TEST] Acc: 45.600
Epoch: 18
[TRAIN] Acc: 81.036
[TEST] Acc: 48.600
Epoch: 19
[TRAIN] Acc: 84.392
[TEST] Acc: 48.040
Epoch: 20
[TRAIN] Acc: 86.934
[TEST] Acc: 46.

[TEST] Acc: 53.870
Epoch: 167
[TRAIN] Acc: 99.978
[TEST] Acc: 53.880
Epoch: 168
[TRAIN] Acc: 99.982
[TEST] Acc: 53.930
Epoch: 169
[TRAIN] Acc: 99.980
[TEST] Acc: 53.860
Epoch: 170
[TRAIN] Acc: 99.990
[TEST] Acc: 53.940
Epoch: 171
[TRAIN] Acc: 99.980
[TEST] Acc: 54.060
Epoch: 172
[TRAIN] Acc: 99.982
[TEST] Acc: 53.660
Epoch: 173
[TRAIN] Acc: 99.982
[TEST] Acc: 53.930
Epoch: 174
[TRAIN] Acc: 99.986
[TEST] Acc: 53.930
Epoch: 175
[TRAIN] Acc: 99.984
[TEST] Acc: 54.010
Epoch: 176
[TRAIN] Acc: 99.978
[TEST] Acc: 53.850
Epoch: 177
[TRAIN] Acc: 99.982
[TEST] Acc: 53.870
Epoch: 178
[TRAIN] Acc: 99.980
[TEST] Acc: 53.990
Epoch: 179
[TRAIN] Acc: 99.984
[TEST] Acc: 53.940
Epoch: 180
[TRAIN] Acc: 99.990
[TEST] Acc: 54.030
Epoch: 181
[TRAIN] Acc: 99.974
[TEST] Acc: 53.890
Epoch: 182
[TRAIN] Acc: 99.982
[TEST] Acc: 53.880
Epoch: 183
[TRAIN] Acc: 99.984
[TEST] Acc: 54.000
Epoch: 184
[TRAIN] Acc: 99.978
[TEST] Acc: 53.970
Epoch: 185
[TRAIN] Acc: 99.974
[TEST] Acc: 53.910
Epoch: 186
[TRAIN] Acc: 99.974


In [15]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

merged=test.merge(train[['Epochs', 'Train Accuracy']], how='left', left_on='Epochs', right_on='Epochs')

merged.to_csv('VGG_CIFAR100_Swish.csv', index=False)

In [16]:
os.remove("train.csv")
os.remove("test.csv")